In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer, TFBertForSequenceClassification, AdamW
from datasets import load_dataset, Dataset
import import_ipynb         # Untuk Import file ipynb
import preprocess

d:\Pemrograman\python\nlp_project\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Pemrograman\python\nlp_project\env\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|██████████| 10000/10000 [00:03<00:00, 2928.25 examples/s]


In [ ]:
# Langkah 3: Memuat Model BERT Pretrained untuk Klasifikasi Sentimen
model_name = preprocess.model_name
tokenized_datasets = preprocess.tokenized_datasets
tokenizer = preprocess.tokenizer

model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=1)

# Langkah 4: Mengonversi dataset ke format TensorFlow
train_dataset = tokenized_datasets['train'].to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols=["label"],
    batch_size=16,
    shuffle=True,       # Mengacak urutan data, untuk mencegah overfitting
)

val_dataset = tokenized_datasets['test'].to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols=["label"],
    batch_size=16,
)

# Langkah 5: Menyiapkan Optimizer, Loss Function dan metrik
optimizer = tf.keras.optimizers.AdamW(learning_rate=2e-5, weight_decay=0.01)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = [tf.keras.metrics.BinaryAccuracy()]

# Kompilasi model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# Langkah 6: Melatih Model
model.fit(train_dataset, validation_data=val_dataset, epochs=3)

In [ ]:
# Langkah 7: Evaluasi Model
results = model.evaluate(val_dataset)
print(f"Validation loss: {results[0]}, Validation accuracy: {results[1]}")

In [ ]:
# Langkah 8: Prediksi
kalimat = "I love this movie!"
inputs = tokenizer(kalimat, return_tensors="tf")
logits = model(inputs).logits
predicted_class = tf.argmax(logits, axis=-1)

print(f"Predicted sentiment: {'Positive' if predicted_class == 1 else 'Negative'}")

In [ ]:
model.save("classification_sentiment_model", save_format="tf")